In [5]:
import pandas as pd 
import os

## Посмотрим на наши файлы и сделаем из них csv

In [6]:
def convert_xlsx_to_csv(directory):
    files = os.listdir(directory)
    
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(directory, file)
            read_file = pd.read_excel(file_path, engine='openpyxl')
            csv_file_path = os.path.join(directory, file.replace('.xlsx', '.csv'))
            read_file.to_csv(csv_file_path, index=False, header=True)
            os.remove(file_path)

In [7]:
convert_xlsx_to_csv('data')

In [21]:
files = os.listdir('data')
file_names = [file.split('.')[0] for file in files]
file_paths = {file_name: file for file_name, file in zip(file_names, list(map(lambda x: 'data/' + x, files)))}
file_paths

{'CHMF': 'data/CHMF.csv',
 '': 'data/.DS_Store',
 'gazp': 'data/gazp.csv',
 'ROSN': 'data/ROSN.csv',
 'SNGSP': 'data/SNGSP.csv',
 'sber': 'data/sber.csv',
 'STSB': 'data/STSB.csv',
 'MGNT': 'data/MGNT.csv',
 'brent_prices': 'data/brent_prices.csv',
 'GMKN': 'data/GMKN.csv',
 'eur_rate': 'data/eur_rate.csv',
 'LPSB': 'data/LPSB.csv',
 'raw_shares_data': 'data/raw_shares_data',
 'usd_rate': 'data/usd_rate.csv',
 'LKOH': 'data/LKOH.csv'}

Ура, теперь все точно csv! Если изначально все файлы были в правильном формате, то функцию все равно можно вызывать, просто ничего не изменится.

## Обрабатываем данные по отдельности

### brent_prices

In [22]:
pd.read_csv('data/brent_prices.csv').head()

,date,price,open,max,min,volume,change
0,2023-12-29,77.04,77.38,77.98,76.73,192.95,-1.72
1,2023-12-28,78.39,79.80,79.95,78.25,18.78,-1.58
2,2023-12-27,79.65,80.79,81.33,79.29,70.39,-1.75
3,2023-12-26,81.07,79.04,81.72,78.76,59.74,2.53
4,2023-12-22,79.07,79.35,80.37,78.88,119.19,-0.40


In [33]:
#это работает с сырыми файлами, сейчас не нужно
# df = pd.read_csv('data/brent_prices.csv')
# df = df.rename(columns={'Дата': 'date', 'Цена': 'price', 'Откр.': 'open', 
#                         'Макс.': 'max', 'Мин.': 'min', 
#                         'Объём': 'volume', 'Изм. %': 'change'})
# df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
# df[['price', 'open', 'max', 'min']] = df[['price', 'open', 'max', 'min']].replace(',', '.', regex=True).astype(float)
# df['volume'] = df['volume'].replace('K', '', regex=True).replace(',', '.', regex=True).astype(float)
# df['change'] = df['change'].replace('%', '', regex=True).str.replace(',', '.', regex=True).astype(float)

# df.to_csv('data/brent_prices.csv', index=False)
df

,date,curs
0,2023-12-30,89.6883
1,2023-12-29,90.3041
2,2023-12-28,91.7051
3,2023-12-27,91.7069
4,2023-12-26,91.9690
...,...,...
980,2020-01-15,61.4140
981,2020-01-14,60.9474
982,2020-01-11,61.2632
983,2020-01-10,61.2340


### eur_rate

In [34]:
df = pd.read_csv('data/eur_rate.csv')
df

,date,curs
0,2023-12-30,89.6883
1,2023-12-29,90.3041
2,2023-12-28,91.7051
3,2023-12-27,91.7069
4,2023-12-26,91.9690
...,...,...
980,2020-01-15,61.4140
981,2020-01-14,60.9474
982,2020-01-11,61.2632
983,2020-01-10,61.2340


In [35]:
# df['nominal'].unique(), df['cdx'].unique()

In [37]:
# df = df.drop(['nominal', 'cdx'], axis=1)
df = df.rename(columns={'data': 'date'})
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['curs'] = df['curs'].astype(float)

df.to_csv('data/eur_rate.csv', index=False)
df

,date,curs
0,2023-12-30,89.6883
1,2023-12-29,90.3041
2,2023-12-28,91.7051
3,2023-12-27,91.7069
4,2023-12-26,91.9690
...,...,...
980,2020-01-15,61.4140
981,2020-01-14,60.9474
982,2020-01-11,61.2632
983,2020-01-10,61.2340


### sber and other shares

In [38]:
df = pd.read_csv('data/sber.csv')
df

,date,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,2020-01-03,255.97,258.19,253.70,255.00,35824160
1,2020-01-06,254.75,254.84,251.40,253.90,22327940
2,2020-01-08,253.62,259.15,253.03,259.15,40736030
3,2020-01-09,259.41,261.76,257.01,257.99,38196300
4,2020-01-10,257.91,259.25,256.88,258.19,18841650
...,...,...,...,...,...,...
988,2023-12-25,271.75,273.85,270.05,271.08,37827910
989,2023-12-26,271.00,272.89,270.00,271.90,22575240
990,2023-12-27,271.90,272.59,270.85,271.08,17191450
991,2023-12-28,270.99,272.48,268.52,271.74,29416450


In [39]:
def preprocess_share_data(path_to_csv, folder_to_save = 'data'):
    """
    Сохраняет нужную информацию по акциям в нужном формате,
      преобразует дату к корректной строке
      делает тип данных
    """
    df = pd.read_csv(path_to_csv)
    df = df.rename(columns={'<DATE>': 'date'})
    df = df.drop(['<TICKER>', '<PER>', '<TIME>'], axis=1)
    #добавляем начало года, чтобы преобразовать к дате
    df['date'] = df['date'].apply(lambda x: '20' + str(x))
    df[[
        '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>',
    ]] = df[[
        '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>',
    ]].astype(float)
    
    new_name = path_to_csv.split('/')[-1].split('_')[0]

    df.to_csv(f"{folder_to_save}/{new_name}.csv", index=False)

In [40]:
from tqdm import tqdm
folder_with_raw_data = '/Users/akzholba/Documents/РискМенеджмент/Проект_2/market_risk/data/raw_shares_data'

for csv_name in tqdm(os.listdir(folder_with_raw_data)):
    preprocess_share_data(f"{folder_with_raw_data}/{csv_name}")

100%|██████████| 8/8 [00:00<00:00, 177.41it/s]


### usd_rate

In [41]:
df = pd.read_csv('data/usd_rate.csv')
df

,date,curs
0,2023-12-30,89.6883
1,2023-12-29,90.3041
2,2023-12-28,91.7051
3,2023-12-27,91.7069
4,2023-12-26,91.9690
...,...,...
980,2020-01-15,61.4140
981,2020-01-14,60.9474
982,2020-01-11,61.2632
983,2020-01-10,61.2340


In [43]:
# df = df.drop(['nominal', 'cdx'], axis=1)
df = df.rename(columns={'data': 'date'})
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['curs'] = df['curs'].astype(float)

df.to_csv('data/usd_rate.csv', index=False)
df

,date,curs
0,2023-12-30,89.6883
1,2023-12-29,90.3041
2,2023-12-28,91.7051
3,2023-12-27,91.7069
4,2023-12-26,91.9690
...,...,...
980,2020-01-15,61.4140
981,2020-01-14,60.9474
982,2020-01-11,61.2632
983,2020-01-10,61.2340


## Проверка, что никакие файлы не потерялись

In [44]:
os.listdir('data')

['CHMF.csv',
 '.DS_Store',
 'gazp.csv',
 'ROSN.csv',
 'SNGSP.csv',
 'sber.csv',
 'STSB.csv',
 'MGNT.csv',
 'brent_prices.csv',
 'GMKN.csv',
 'eur_rate.csv',
 'LPSB.csv',
 'raw_shares_data',
 'usd_rate.csv',
 'LKOH.csv']

In [59]:
# del file_paths['']
del file_paths['raw_shares_data']

In [60]:
for name, path in file_paths.items():
    print(name)
    print(pd.read_csv(path, header = 0).columns)
    print('-'*10)

CHMF
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
----------
gazp
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
----------
ROSN
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
----------
SNGSP
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
----------
sber
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
----------
STSB
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
----------
MGNT
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
----------
brent_prices
Index(['date', 'price', 'open', 'max', 'min', 'volume', 'change'], dtype='object')
----------
GMKN
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>'], dtype='object')
----------
eur_rate
Index(['date', 'curs'], dtype='object')
----------
LPSB
Index(['date', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE

Ура, эти 5 файлов немного высосали из меня силы, но зато теперь можно тестить второе задание и использовать этот ноутбук как пример для работы со следующими риск факторами.